In [5]:
!pip install transformers peft datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [6]:
!pip install accelerate -U

In [7]:
pip install transformers[torch]

In [8]:
pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.8 MB/s eta 0:00:00


In [9]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [10]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00


In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [2]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Direct to your assignment folder.
%cd /content/drive/MyDrive/project-m2-2024-jim

/content/drive/MyDrive/project-m2-2024-jim


In [ ]:
import torch
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer
#from transformers.utils.quantization_config import BitsAndBytesConfig
from datasets import load_dataset, DatasetDict
from trl import DPOTrainer
from peft import LoraConfig, get_peft_model
import torch
import wandb, os
import warnings

# Ignore specific warning categories or use 'ignore' to suppress all warnings
#warnings.filterwarnings("ignore", category=UserWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)


#os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
import json
from datasets import Dataset
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def load_and_preprocess_jsonl(file_path):
    # Load the preference dataset from JSONL file
    def load_jsonl(file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()
            data = [json.loads(line) for line in lines]
        return data

    preference_data = load_jsonl(file_path)

    # Convert to Hugging Face Dataset
    preference_dataset = Dataset.from_list(preference_data)

    # Ensure prompt, chosen, and rejected fields are strings
    def preprocess_preference_data(example):
        return {
            "prompt": example["prompt"],
            "chosen": example["chosen"],
            "rejected": example["rejected"]
        }

    # Apply preprocessing to the preference dataset
    preference_dataset = preference_dataset.map(preprocess_preference_data)

    return preference_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Load and preprocess the first preference dataset
preference_dataset_1 = load_and_preprocess_jsonl("data/intel_dataset.jsonl")


Map:   0%|          | 0/64033 [00:00<?, ? examples/s]

In [ ]:

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.tensorboard import SummaryWriter


## setting up the models
token = os.getenv('HF_TOKEN') ## set token
# Load the fine-tuned model from the SFT step
model_output_dir = "./model_SFT_final"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model.config.use_cache = False

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['c_attn', 'c_proj', 'mlp.c_fc', 'mlp.c_proj']  # Targeting GPT-2 specific modules
    )
model = get_peft_model(model, peft_config)


# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir="./model_output_dpo",
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="tensorboard",
)

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter(log_dir='./model_output_dpo')


## setup trainer
max_length = 1024
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=preference_dataset_1,
    #eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    loss_type = 'sigmoid',
    max_prompt_length= max_length,
    max_length= max_length,
)

## Start training
dpo_trainer.train()


# Save the model and tokenizer after the first DPO training
model_output_dir_dpo_1 = "./model_output_dpo_1"
dpo_trainer.model.save_pretrained(model_output_dir_dpo_1)
tokenizer.save_pretrained(model_output_dir_dpo_1)

Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1105 > 1024). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.751100
4,0.732200
5,0.687900
6,0.747300
7,0.728800
8,0.795500
9,0.696700
10,0.793200


Step,Training Loss
1,0.693100
2,0.693100
3,0.751100
4,0.732200
5,0.687900
6,0.747300
7,0.728800
8,0.795500
9,0.696700
10,0.793200


('./model_output_dpo_1/tokenizer_config.json',
 './model_output_dpo_1/special_tokens_map.json',
 './model_output_dpo_1/vocab.json',
 './model_output_dpo_1/merges.txt',
 './model_output_dpo_1/added_tokens.json')

In [ ]:
# Load and preprocess the first preference dataset
preference_dataset_2 = load_and_preprocess_jsonl("data/collaborative_dataset.jsonl")


Map:   0%|          | 0/26738 [00:00<?, ? examples/s]

In [ ]:

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.tensorboard import SummaryWriter


## setting up the models
token = os.getenv('HF_TOKEN') ## set token
# Load the fine-tuned model from the previous step
model_output_dir = "./model_output_dpo_1"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['c_attn', 'c_proj', 'mlp.c_fc', 'mlp.c_proj']  # Targeting GPT-2 specific modules
    )
model = get_peft_model(model, peft_config)


# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir="./model_output_dpo_2",
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="tensorboard",
)

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter(log_dir='./model_output_dpo_2')


## setup trainer
max_length = 1024
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=preference_dataset_2,
    #eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    loss_type = 'sigmoid',
    max_prompt_length= max_length,
    max_length= max_length,
)

## Start training
dpo_trainer.train()


# Save the model and tokenizer after the first DPO training
model_output_dir_dpo_2 = "./model_output_dpo_2"
dpo_trainer.model.save_pretrained(model_output_dir_dpo_2)
tokenizer.save_pretrained(model_output_dir_dpo_2)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/26738 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 1024). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.744900
4,0.790500
5,0.756800
6,0.756300
7,0.646200
8,0.719100
9,0.697400
10,0.656500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./model_output_dpo_2/tokenizer_config.json',
 './model_output_dpo_2/special_tokens_map.json',
 './model_output_dpo_2/vocab.json',
 './model_output_dpo_2/merges.txt',
 './model_output_dpo_2/added_tokens.json',
 './model_output_dpo_2/tokenizer.json')

In [ ]:
# Load and preprocess the first preference dataset
preference_dataset_3 = load_and_preprocess_jsonl("data/argilla_dataset.jsonl")


Map:   0%|          | 0/63619 [00:00<?, ? examples/s]

In [ ]:

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.tensorboard import SummaryWriter


## setting up the models
token = os.getenv('HF_TOKEN') ## set token
# Load the fine-tuned model from the previous step
model_output_dir = "./model_output_dpo_2"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['c_attn', 'c_proj', 'mlp.c_fc', 'mlp.c_proj']  # Targeting GPT-2 specific modules
    )
model = get_peft_model(model, peft_config)


# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir="./model_output_dpo_3",
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="tensorboard",
)

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter(log_dir='./model_output_dpo_3')


## setup trainer
max_length = 1024
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=preference_dataset_3,
    #eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    loss_type = 'sigmoid',
    max_prompt_length= max_length,
    max_length= max_length,
)

## Start training
dpo_trainer.train()


# Save the model and tokenizer after the first DPO training
model_output_dir_dpo_3 = "./model_output_dpo_3"
dpo_trainer.model.save_pretrained(model_output_dir_dpo_3)
tokenizer.save_pretrained(model_output_dir_dpo_3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/63619 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 1024). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.648300
4,0.738200
5,0.683900
6,0.703600
7,0.747000
8,0.720600
9,0.697900
10,0.872000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./model_output_dpo_3/tokenizer_config.json',
 './model_output_dpo_3/special_tokens_map.json',
 './model_output_dpo_3/vocab.json',
 './model_output_dpo_3/merges.txt',
 './model_output_dpo_3/added_tokens.json',
 './model_output_dpo_3/tokenizer.json')

In [ ]:
# Load and preprocess the first preference dataset
preference_dataset_nectar = load_and_preprocess_jsonl("data/nectar_train_dataset.jsonl")


Map:   0%|          | 0/64033 [00:00<?, ? examples/s]

In [ ]:

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.tensorboard import SummaryWriter


## setting up the models
token = os.getenv('HF_TOKEN') ## set token
# Load the fine-tuned model from the previous step
model_output_dir = "/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_3"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['c_attn', 'c_proj', 'mlp.c_fc', 'mlp.c_proj']  # Targeting GPT-2 specific modules
    )
model = get_peft_model(model, peft_config)


# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir="/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar",
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="tensorboard",
)

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter(log_dir='/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar')


## setup trainer
max_length = 1024
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=preference_dataset_3,
    #eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    loss_type = 'sigmoid',
    max_prompt_length= max_length,
    max_length= max_length,
)

## Start training
dpo_trainer.train()


# Save the model and tokenizer after the first DPO training
model_output_dir_dpo_nectar = "/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar"
dpo_trainer.model.save_pretrained(model_output_dir_dpo_nectar)
tokenizer.save_pretrained(model_output_dir_dpo_nectar)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/63619 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 1024). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.696800
4,0.709700
5,0.708400
6,0.688700
7,0.728100
8,0.759100
9,0.689900
10,0.829000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/tokenizer_config.json',
 '/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/special_tokens_map.json',
 '/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/vocab.json',
 '/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/merges.txt',
 '/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/added_tokens.json',
 '/content/drive/MyDrive/project-m2-2024-jim/checkpoints/model_output_dpo_nectar/tokenizer.json')

In [28]:
!python evaluator.py

^C
